# Wheelchair detection training 

In this project, I present a Faster RCNN-based wheelchair detection system. This code performs the training and mAP accuracy measurements. 

First, modules: 

In [1]:
from __future__ import division
import random
import pprint
import sys
import time
import numpy as np
import pickle
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score
from optparse import OptionParser
from keras import backend as K
from keras.optimizers import Adam #, SGD, RMSprop
from keras.layers import Input
from keras.models import Model
from keras_frcnn import config, data_generators
from keras_frcnn import losses as losses
from keras.utils import generic_utils, plot_model 
import keras_frcnn.roi_helpers as roi_helpers
sys.setrecursionlimit(40000)

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


Specifying parser options

In [ ]:
parser = OptionParser()

parser.add_option("-p", "--path", dest="train_path", help="Path to training data.")

parser.add_option("-o", 
                 "--parser", 
                  dest="parser", 
                  help="Parser to use. One of simple or pascal_voc",
                  default="pascal_voc")

parser.add_option("-n", 
                  "--num_rois", 
                  dest="num_rois", 
                  help="Number of RoIs to process at once.", 
                  default=32)

parser.add_option("--network", 
                  dest="network", 
                  help="Base network to use. Supports vgg or resnet50.", 
                  default='resnet50')

parser.add_option("--hf", 
                  dest="horizontal_flips", 
                  help="Augment with horizontal flips in training. (Default=false).", 
                  action="store_true", 
                  default=False)

parser.add_option("--vf", 
                  dest="vertical_flips", 
                  help="Augment with vertical flips in training. (Default=false).", 
                  action="store_true", 
                  default=False)

parser.add_option("--rot", "--rot_90", 
                  dest="rot_90", 
                  help="Augment with 90 degree rotations in training. (Default=false).",
                  action="store_true", 
                  default=False)

parser.add_option("--num_epochs", 
                  dest="num_epochs", 
                  help="Number of epochs.", 
                  default=2000)

parser.add_option("--config_filename", 
                  dest="config_filename",
                  help="Location to store all the metadata related to the training (to be used when testing).",
                  default="config.pickle")

parser.add_option("--output_weight_path", 
                  dest="output_weight_path", 
                  help="Output path for weights.", 
                  default='./model_frcnn.hdf5')

parser.add_option("--input_weight_path", 
                  dest="input_weight_path", 
                  help="Input path for weights. If not specified, will try to load default weights provided by keras.")

(options, args) = parser.parse_args()
